In [1]:
# CFV Extrate                                 \\   Finish 

# Pre process
    #  1) delete all return statements then add it in the end of function

# Select Vulnerability Location               \\   Finish
    #  1) change the allocate weight method   \\   Finish
    #  2) allocate the magic number weight    \\   Finish
    #  3) add one more parameter to the function
    #     to make the path unique             \\   Finish

# Dataflow Marker                             \\   Finish 
    #  1) change the definition parameters    \\   Finish  
    
# Vulnerability Insertion                     \\   Finish
    #  1) use the track to insert vul code    \\   Finish
      
# Function Call & Controller Insertion        \\   Finish
    #  1) checksum parameter controller       \\   Finish    
    #  2) magic number length parameter       \\   Finish    
    #  3) change input type to "scanf"        \\   Finish  
    #  4）change the comparation expression   \\   Finish
    
# Key Words Statements Insertion              \\   Finish
    #  1) change the loop and if statements   \\   Finish
    #  2) fix the bug                         \\   Finish 
    #  3) add the magic parameter controller  \\   Finish
    #  4）add function call                   \\   Finish
    #  5) more key words processor            \\   Finish
    #  6) add if numbers controller parameter \\   Finish
    #  7) remove some special symbols         \\   Finish 
    
# Environment Code Generation                 \\   Finish
    #  1) add unique function call processor  \\   Finish   
    
# Formalization                               \\   Finish
    #  1) add includings                      \\   Finish
    #  2) add vul lib includings              \\   Finish

# Known Bug:
#    1) the optional question in environment code generation
#    2) the function calls may be inserted in front of the dataflow     (Fixed)

# TO DO :
#    1) add fake vulberability 

In [2]:
import os
from xml.dom.minidom import parse
import xml.dom.minidom
import numpy as np
import random
import csv
import networkx as nx
import unicodedata
import copy

import CFV_Extractor as CE
import Select_Vul_Location as VL
import FuncCall_Controller_Insertion as FCI
import Vul_Marker as VM
import Vul_Insertion as VI
import Environment_Code_Generator as ECG
import Environment_Key_Statement_Insertion as EKSI
import Formalization as Formal


GrammarTree = r"./GrammarTree.xml" # the grammar tree path
VulLib = r"./VulnerabilityLib.xml" # the vulnerability lib path

FilePath = "./biscuit_1.gpickle"
gpickleCSV = "./gpickle.csv"

In [3]:
# ================ ulit parameters =======================

# the unique path check number
unique_magic_number = random.randint(0,1000)

# the chose vulnerability name
Vul_name = "CWE-761"

# dataflow parameters and environment parameters

# ================ dataflow parameters =====================

# the input number indicte the deepth of the vulnerability
# e.g. ' input_number = 10 ' means the vulnerability was hiden in 
#       a 10 deepth input nest (if you want to trige the vulnerability,
#       you have to pass the 10 if-input-checks)
#  the input check looks like ' if( strcmp(controller_1,"abcd") > 0)'
dataflow_parameter_input_number = 10            
            
# the magic number based on input numbers
# e.g. ' magic_number = 1 '
#       if the input_number equals 10, one of them will become a 
#       magic number check
#  the magic number check looks like ' if( strcmp(controller_1,"abcd") == 0) ' 
dataflow_parameter_magic_number = 1           
            
            
# the magic_number_mix and max limit the input length
# for e.g. ' min = 1 && max = 5 ' means the length of the magic number check will be randonly picked from 1 to 5
dataflow_parameter_magic_number_length_min = 1
dataflow_parameter_magic_number_length_max = 5

# the checksum number based on magic numbers
# e.g. ' checksum = 1 ' 
#       if the input_number equals 10, one of them will become a 
#       checksum check
dataflow_parameter_checksum = 0
# the type number of checksum
# 1 = remainder_check
# 2 = .....(to be continued)
# the checksum check look like ' if(remainder_check(controller_1,checksum_div,length_of_controller_1) '
# more detail about the checksum function , please see 'Formalization.py'
dataflow_parameter_checksum_type = 1
# the dividend for remainder_checksum
dataflow_parameter_checksum_div = 100



# ================ environment parameters =====================

# branches means the input check numbers in environment code
# e.g. ' branches = 100 ' means there are 100 input checks in the environment code
environment_parameter_branches = 10

environment_parameter_magic_number_length_min = 1  # same meaning with dataflow one but in environment
environment_parameter_magic_number_length_max = 5  # same meaning with dataflow one but in environment
environment_parameter_magic_number = 0             # same meaning with dataflow one but in environment

environment_parameter_checksum_type = 1     # same meaning with dataflow one but in environment
environment_parameter_checksum_div = 100    # same meaning with dataflow one but in environment
environment_parameter_checksum_number = 0   # same meaning with dataflow one but in environment

# control loop time of all the loopers in environment code
environment_paramenter_loop_time = 1


In [4]:
# abandoned function
# to test the function callee and caller`s relationship in their refids and refedids
def recover_test():
    for i in range(1,len(selectedPath)):
        if nodeSet[selectedPath[i]]["id"] in nodeSet[selectedPath[i-1]]["refids"]:
            print("1")
        else:
            print("2")

    for i in range(1,len(selectedPath)):
        if nodeSet[selectedPath[i-1]]["id"] in nodeSet[selectedPath[i]]["refedids"]:
            print("3")
        else:
            print("4")

In [5]:
# test-use only
# display the vulnerablity related data:
#     1) vul_path
#     2) nest_path_weight
#     3) maigc_number_weight
#     4) checksum_weight
def display_vul_path():
    for i in range(0,len(selectedPath)):
        print(nodeSet[selectedPath[i]]["name"])
        print(nodeSet[selectedPath[i]]["cfv"])
        print("==================================")
    print("vul_path:")
    print(selectedPath)
    print("path_weight:")
    print(selectedWeight)
    print("magic number weight")
    print(magicWeight)
    print("checksum weight")
    print(checksumWeight)

In [6]:
# get all hyper parameters and organize them in a txt content format
def getParameters():
    
    content = ""
    
    # ulit parameters
    content += "\n\n================= ulit parameters ======================\n\n"
    content += "unique_magic_number = "+ str(unique_magic_number)+"\n"
    content += "vulnerability_name = "+ str(Vul_name)+"\n"
    content += "\n================= ulit parameters ends ======================"
    
    # dataflow parameters
    content +="\n\n================= dataflow parameters ======================\n\n"
    content +="dataflow_parameter_input_number = "+str(dataflow_parameter_input_number)+"\n\n"
    content +="dataflow_parameter_magic_number = "+str(dataflow_parameter_magic_number)+"\n"
    content +="dataflow_parameter_magic_number_length_min = "+str(dataflow_parameter_magic_number_length_min)+"\n"
    content +="dataflow_parameter_magic_number_length_max = "+str(dataflow_parameter_magic_number_length_max)+"\n\n"
    content +="dataflow_parameter_checksum_type = "+str(dataflow_parameter_checksum_type)+"\n"
    content +="dataflow_parameter_checksum_div = "+str(dataflow_parameter_checksum_div)+"\n"
    content +="dataflow_parameter_checksum = "+str(dataflow_parameter_checksum)+"\n"
    content +="\n================= dataflow parameters ends ======================"

    # environment parameters
    content +="\n\n================= environment parameters ======================\n\n"
    content +="environment_parameter_branches = "+str(environment_parameter_branches)+"\n\n"
    content +="environment_parameter_magic_number_length_min = "+str(environment_parameter_magic_number_length_min)+"\n"
    content +="environment_parameter_magic_number_length_max = "+str(environment_parameter_magic_number_length_max)+"\n"
    content +="environment_parameter_magic_number = "+str(environment_parameter_magic_number)+"\n\n"
    content +="environment_parameter_checksum_type = "+str(environment_parameter_checksum_type)+"\n"
    content +="environment_parameter_checksum_div = "+str(environment_parameter_checksum_div)+"\n"
    content +="environment_parameter_checksum_number = "+str(environment_parameter_checksum_number)+"\n\n"
    content +="environment_paramenter_loop_time = "+str(environment_paramenter_loop_time)+"\n"
    content +="\n================= environment parameters ends ======================"
    
    return content

In [7]:
# filter for the word list
# filter all keyword in 'filter_list'
# parameter: nodeSet -> which contains all the function node
def pre_process_filter(nodeSet):
    filter_list = ["switch","case","?:","break","default","continue"]
    for i in range(0,len(nodeSet)):
        index = 0
        while index<len(nodeSet[i]["cfv"]):
            if nodeSet[i]["cfv"][index] in filter_list:
                del nodeSet[i]["cfv"][index]
            else:
                index+=1
        
    return nodeSet
                
# rename the confilcted function name
# add a prefix in the confilcted name and change the name in all reference or referenced call
# parameter: nodeSet -> which contains all the function node
def pre_process_func_name(nodeSet):
    func_name_list = []
    for i in range(0,len(nodeSet)):
        if nodeSet[i]["name"] not in func_name_list:
            func_name_list.append(nodeSet[i]["name"])
        else:
            new_name = "new_"+nodeSet[i]["name"]
            old_name = nodeSet[i]["name"]
            retType = nodeSet[i]["full_definition"].split(nodeSet[i]["name"])[0].strip()
            
            nodeSet[i]["full_definition"] = nodeSet[i]["full_definition"].replace(old_name,new_name)
            nodeSet[i]["name"] = new_name 
            
            # replace the old name in refDef of other nodes with new name
            for index in range(0,len(nodeSet)):
                for j in range(0,len(nodeSet[index]["refDef"])):
                    if nodeSet[index]["refDef"][j][0] == old_name:
                        if nodeSet[index]["refDef"][j][1].split(old_name)[0].strip()==retType:
                            refDef = (new_name,nodeSet[index]["refDef"][j][1].replace(old_name,new_name))
                            nodeSet[index]["refDef"][j] = refDef
                            
    return nodeSet

# add return to the blank cfv node
# if the blank cfv node needs a return variable, the cfv turns to ['{','return','}']
# if the blank cfv node doesn`t have a return variable, the cfv turns to ['{',';','}']
# parameter: nodeSet -> which contains all the function node
def pro_process_blank_cfv(nodeSet):
    for i in range(0,len(nodeSet)):
        if len(nodeSet[i]["cfv"])==0:
            #print(nodeSet[i]["full_definition"])
            if nodeSet[i]["full_definition"].startswith("void"):
                nodeSet[i]["cfv"] = ['{',';','}']
            else:
                nodeSet[i]["cfv"] = ['{','return','}']
    return nodeSet

In [8]:
csv_reader = csv.reader(open(gpickleCSV,'r'))
gpickle_list = []
for row in csv_reader:
    row[0] = row[0].replace("\\","/")
    gpickle_list.append(row)

del gpickle_list[0]

In [9]:
poss_list = []
for gpickle in gpickle_list:
    if int(gpickle[1]) >= 10 and int(gpickle[2]) >= 10:
        poss_list.append(gpickle[0])

In [10]:
FilePath = poss_list[random.randint(0,len(gpickle)-1)]
print(FilePath)

./gpickle/aws-iot-device-sdk-embedded-C_0.gpickle


In [12]:
# =================================== extract function nodes and pre process the nodes ===================================

In [13]:
# get the node list from gpickle file 
extractor = CE.CFV_Extractor(FilePath)
nodeSet = extractor.getAllCFV()

In [14]:
nodeSet = pre_process_filter(nodeSet)  # pre process for filtering some keywords
nodeSet = pre_process_func_name(nodeSet) # pre process for renameing the confilcted functions
nodeSet = pro_process_blank_cfv(nodeSet) # pre process for adding content into the blank cfv

In [15]:
# =================================== extract function nodes and pre process the nodes ends ===================================

In [17]:
# =================================== locate the vul location ===================================

In [18]:
# via the customized hyper parameter, find a suitable place for vulnerability
locator = VL.Select_Vul_Location(extractor                         # the cfv extrotor
                                 ,dataflow_parameter_input_number  # the hyper parameter - input number
                                 ,dataflow_parameter_magic_number  # the hyper parameter - magic number
                                 ,dataflow_parameter_checksum)     # the hyper parameter - checksum

# the vul_path

# e.g. vul_path = [0,10,11,12]
#      it means that to trige the vulnerability, you have to pass the No.0 node, No.10 node and No.12 node
selectedPath = locator.getSelectedPath()

# the nest_weight
# it should be used with vul_path and the index of the list indicate the index in vul_path

# e.g. nest_weight = [2,1,0,1] && vul_path = [0,10,11,12]

# the nest_weight[0] = 2 means in the No.0 node (vul_path[0]), we call the No.10 node (vul_path[1]) function in a 
#   two-level (nest_weight[0] = 2 ) if nest.
# the nest_weight[1] = 1 means in the No.10 node (vul_path[1]), we call the No.11 node (vul_path[2]) function in a 
#   one-level (nest_weight[1] = 1 ) if nest.
#  ````````
# the nest_weight[3] = 1 means in the No.12 node (vul_path[1]), we trige the vulnerability in a one-level 
#   (nest_weight[3] = 1) if nest
selectedWeight = locator.getSelectedWeight()

# the magic_number_weight
# it was conducted by hyper parameter maigc number and the nest_weight

# e.g. magic_number_weight = [1,1,0,0] && nest_weight = [2,1,0,1]
# the magic number weight is based on the nest_weight
# magic_number_weight[0] = 1 means in the 2 input checks (nest_weight[0]=2), the 1 of them will become magic number check
magicWeight = locator.getMagicNumberWeight()

# the checksum_weight
# it was conducted by hyper parameter checksum and the nest_weight

# e.g. checksum_weight = [1,1,0,0] && magic_number_weight = [2,1,0,1]
# the checksum_weight is based on the magic_number_weight
# checksum_weight[0] = 1 means in the 2 magic number check (magic_number_weight[0]=2), the 1 of them will become checksum
checksumWeight = locator.getChecksumWeight()

In [19]:
# test use only
# locator.displayOutput()

In [20]:
# =================================== locate the vul location Ends ===================================

In [21]:
# =================================== vul location marker ===================================

In [22]:
# add the dataflow marks into the node list
nodeSet = VM.markDataflow(nodeSet,selectedWeight,selectedPath) 

In [23]:
# change the definition of functions which in dataflow
# add a new parameter named 'unique_parameter' to control that there is only one way to trige the vulnerability
nodeSet = VM.changeDefinitions(nodeSet,selectedPath)

In [24]:
# =================================== vul location marker Ends ===================================

In [25]:
# =================================== vul insertion ===================================

In [26]:
# get the vul statement from vul lib xml file
vul_node = VI.getVulNode(Vul_name,VulLib)
# get the variable definition from the xml node
vul_definition = VI.getVulDefinition(vul_node) 
# get the vulnerability statements from the xml node
vul_statement,key_index = VI.getVulStatement(vul_node)

In [27]:
# get the vul track via the marked node and vulnerability statement
vul_track = VI.getVulTrack(nodeSet[selectedPath[-1]]["cfv"],  # the marked node (the place to insert the vulnerability)
                           vul_statement, # vulnerability satatement 
                           key_index)     # indicate the key line in statements 

In [28]:
# use the track to insert the vul code
nodeSet[selectedPath[-1]]["cfv"] = VI.insertStatementByTrack(vul_track,
                                                            nodeSet[selectedPath[-1]]["cfv"],
                                                            unique_magic_number)
# insert the variable definition in the node
nodeSet[selectedPath[-1]]["cfv"] = VI.insertDefinitions(nodeSet[selectedPath[-1]]["cfv"],
                                                       vul_definition)

In [29]:
# =================================== vul insertion Ends ===================================

In [30]:
# =================================== insert function call and controller ===================================

In [31]:
# insert the function call
nodeSet = FCI.insertFunctionCall(nodeSet,selectedPath,unique_magic_number)

In [32]:
number = 0  # the index of magic number weight

# from the vul_path, insert the function call, controller and useless controller to function nodes
for index in selectedPath:
    # get the positions of all keywords in the function node 
    position = FCI.locateController(nodeSet[index]["cfv"])
    
    # if_list -> there are len(if_list) if conditions in the function
    #            each number in if_list the corresponding if condition`s status
    # status ->  0  ==  normal input comparison  e.g. ' if( strcmp(controller4vul ,"abcd") > 0 ) '
    #            1  ==  magic number input comparison  e.g.  ' if( strcmp(controller4vul ,"abcd") == 0 ) '
    #            2  ==  checksum input comparison   e.g.  ' if(remainder_check(controller4vul , checksum_div, length)) '
    if_list = [0]*selectedWeight[number]   
    
    # ----------- get the magic number list ------------
    temp = magicWeight[number]   # temp is used to record the random times
    while temp != 0:  
        index_magic = random.randint(0,selectedWeight[number]-1)  # randomly pick a if block in the if_list 
        if  if_list[index_magic] == 0:    # if the randomly picked if block is a normal input comparison
            temp-=1                       
            if_list[index_magic]+=1       # make the randomly picked if block become a magic number check
    # ----------- get the magic number list ends ------------
    
    # ----------- get the checksum list  ------------
    temp = checksumWeight[number] # temp is used to record the random times
    while temp != 0:
        index_magic = random.randint(0,selectedWeight[number]-1) # randomly pick a if block in the if_list 
        if  if_list[index_magic] == 1:   # if the randomly picked if block is a magic number input comparison
            temp-=1
            if_list[index_magic]+=1      # make the randomly picked magic number check become a checksum input comparison
    # ----------- get the checksum list ends ------------
    
    nodeSet[index] = FCI.insertController(nodeSet[index],                               # the function node 
                                          position,                                     # the keywords positions
                                          dataflow_parameter_magic_number_length_min,   # min of the magic number length 
                                          dataflow_parameter_magic_number_length_max,   # max of the magic number length 
                                          if_list,                                      # the if block list with status
                                          dataflow_parameter_checksum_type,             # the checksum type
                                           dataflow_parameter_checksum_div)             # the checksum div (for remainder_check)
    
    # make if block which is in front of dataflow become useless if block   e.g. ' if(1) '
    nodeSet[index]["cfv"] = FCI.uselessIfInsertion(nodeSet[index]["cfv"])
    
    number+=1

In [33]:
# =================================== insert function call and controller Ends ===================================

In [34]:
# =================================== environment key statement insertion generation ===================================

In [35]:
# scan the if blocks from environment
# ps: the if blocks which have been processed by insert_functionCall_and_controller phase
#     so they cannot be found by this function
def scanIFinEnvironment(nodeSet):
    node_if_numbers = [0] *  len(nodeSet)
    for index in range(0,len(nodeSet)):
        cfv = nodeSet[index]["cfv"]
        for i in range(0,len(cfv)):
            if cfv[i] == "if" or cfv[i] == "else if":
                node_if_numbers[index] += 1
    return node_if_numbers

# allocate the weight
def allocateNumbers(number_lsit,limit_times):
    result_list = [0] * len(number_lsit)
    temp = limit_times      # temp is used to record the random times
    while temp != 0:
        index = random.randint(0,len(number_lsit)-1) # randomly pick a if block in the if_list 
        if  result_list[index] < number_lsit[index]: # if the randomly picked if block status level is smaller than expected
            temp-=1
            result_list[index]+=1                    # make the randomly picked if block level up
    
    return result_list

# change the structure
# e.g.  source_list [3,0,1]  target_list [1,0,1]
#       ouput: result_list [[0,1,0], [], [1]]
def changeNumber2List(source_list,target_list):
    result_list = []
    for i in range(0,len(source_list)):
        temp_list = [0] * source_list[i] 
        limit = target_list[i]  # limit is used to record the random times
        while limit != 0:
            index = random.randint(0,source_list[i]-1)  # randomly pick a if block in the if_list 
            if temp_list[index] == 0:                   # if the randomly picked if block is a useless if condition
                limit-=1
                temp_list[index]+=1                     # make the randomly picked if block level up
        result_list.append(temp_list)
    return result_list

# record the index of if block which is "level" status and in if block list
def recordBranchIndex(branch_list,level):
    result_list = []
    for i in range(0,len(branch_list)):
        for j in range(0,len(branch_list[i])):
            if branch_list[i][j]==level:
                result_list.append([i,j])
    return result_list

# according to the record, add the 'level' status into if block list
def addNumberIntoBranch(branch_list,level,limit,record):
    result_list = copy.copy(branch_list)
    temp_list = [level] * len(record)
    
    temp = limit
    while temp!=0:
        index = random.randint(0,len(temp_list)-1)
        if temp_list[index]==level:
            temp-=1
            temp_list[index]+=1
    
    for index in range(0,len(record)):
        result_list[record[index][0]][record[index][1]] = temp_list[index]
    
    return result_list

In [36]:
# scan the if block in environment
node_if_numbers = scanIFinEnvironment(nodeSet)

# allocate the branches in if block list
branch_list = allocateNumbers(node_if_numbers,environment_parameter_branches)

# change the branch list structure
branch_list = changeNumber2List(node_if_numbers,branch_list)

# record the mangic number if block index in branch list
record4magic_number = recordBranchIndex(branch_list,1)

# according to the magic number record, add the status into the corresponding position in branch list
branch_list = addNumberIntoBranch(branch_list,1,environment_parameter_magic_number,record4magic_number)

# record the checksum if block index in branch list
record4checksum = recordBranchIndex(branch_list,2)

# according to the checksum record, add the status into the corresponding position in branch list
branch_list = addNumberIntoBranch(branch_list,2,environment_parameter_checksum_number,record4checksum)

In [37]:
for index in range(0,len(nodeSet)):
    # func_flag -> record that the function node is in vul_path or not
    #             1  ==  function node is in vul_path and it needs to be inserted function calls
    #             0  ==  function node is not in vul_path and it dones`t need to be inserted function calls
    func_flag = 1
    if index in selectedPath:
        func_flag = 0
    # insert the function calls, controllers and loopers in function node
    statementInserter = EKSI.StatementInserter(nodeSet[index],                           # the function node
                                          environment_paramenter_loop_time,              # loop times
                                          branch_list[index],                            # the if status block
                                          environment_parameter_magic_number_length_min, # min of maigc number length 
                                          environment_parameter_magic_number_length_max, # max of maigc number length 
                                          environment_parameter_checksum_type,           # the checksum type
                                          environment_parameter_checksum_div,            # the checksum dividend
                                          func_flag)                                     # the function node is in vul_path
    # get the processed cfv
    nodeSet[index]["cfv"] = statementInserter.getCFV()
    del statementInserter

In [38]:
# =================================== environment key statement insertion generation Ends ===================================

In [39]:
# =================================== environment code generation ===================================

In [40]:
# insert the normal code expressions into each function node
for index in range(0,len(nodeSet)):
    generator = ECG.CodeGenerator(GrammarTree,         # the GrammarTree xml file path
                                  nodeSet,             # the whole function nodes
                                  index,               # the index of target function node
                                  selectedPath,        # the vul_path
                                  unique_magic_number) # the number to keep vulerable dataflow unique
    nodeSet[index]["cfv"] = generator.getCFV()

In [41]:
# =================================== environment code generation Ends ===================================

In [42]:
# =================================== formalization ===================================

In [43]:
# parse the gpickle file path then generate the track txt file path
trackPath = ""
if "/" in FilePath:
    filename = FilePath.split("/")[-1].split(".")[0]
    trackPath = "./dataset/"+filename +".txt"
    
with open(trackPath,"w",encoding="utf-8") as f:
    f.truncate()
    
    # write the track
    track = Formal.getTrackContent(selectedPath,nodeSet)
    f.write(track)
    
    # write the dataflow and environment parameters
    parameter_content = getParameters()
    f.write(parameter_content)


In [44]:
# clean the vulnerability mark in the function node which is the last one in vul_path
nodeSet[selectedPath[-1]]["cfv"] = Formal.cleanVulMark(nodeSet[selectedPath[-1]]["cfv"])
# add the uni_para claim statement into main function node
nodeSet[0]["cfv"].insert(1,"int uni_para ="+str(unique_magic_number)+";")

In [45]:
# parse the gpickle file path then generate the C file path
CPath = ""
if "/" in FilePath:
    filename = FilePath.split("/")[-1].split(".")[0]
    CPath = "./dataset/"+filename +".c"
    
# write the C codes into C file
with open(CPath,"w",encoding="utf-8") as f:
    f.truncate()
    
    # get the header infomation
    #    ->  the includings  e.g.  #include<stdio.h>
    header_content = Formal.header4program(GrammarTree,vul_node)
    f.write(header_content)
    
    # write the checksum function claim and fucntion body
    checksum = Formal.switchForChecksum(dataflow_parameter_checksum_type)
    f.write(checksum[0])
    f.write(checksum[1])
    if dataflow_parameter_checksum_type != environment_parameter_checksum_type:
        checksum = Formal.switchForChecksum(environment_parameter_checksum_type)
        f.write(checksum[0])
        f.write(checksum[1])
    
    # write all functions claim 
    # main function doesn`t need a claim so it`s range(len(nodeSet-1),0,-1) not range(len(nodeSet-1),-1,-1)
    for index in range(len(nodeSet)-1,0,-1):
        definition = Formal.Definition2Code(nodeSet,index)+";\n"
        f.write(definition)
    
    # write all functions body
    for index in range(len(nodeSet)-1,-1,-1):
        content = Formal.Node2Code(nodeSet,index)
        f.write(content)

In [46]:
# =================================== formalization Ends ===================================